In [ ]:
import tkinter as tk
import re

import ipynb_importer
from publish_subscribe import Reader

In [ ]:
class MainPanel:
    def __init__(self, reader, publisher):
        self.reader = reader
        self.publisher = publisher
        self.all_authors = ['李白', '杜甫', '王維', '白居易']
        self.cur_author = ""

    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('400x400')
        self.root.title("古诗订阅")
        # self.root.iconbitmap('cat.ico')
        # self.root.iconbitmap('@cat.xbm')
        self.label = tk.Label(self.root, text="我的订阅",
                              font=(None, 12)).pack(pady=20)
        self.add_button = tk.Button(
            self.root,
            text="新增订阅",
            font=(None, 12),
            command=self.add_subscription).pack(pady=40)
        self.root.mainloop()

    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('400x300')
        # self.add_panel.iconbitmap('cat.ico')
        # self.root.iconbitmap('@cat.xbm')
        present_author_label = tk.Label(self.add_panel,
                                        text=f"已订阅作者：{self.reader.authors}",
                                        font=(None, 10)).pack(pady=20)
        label = tk.Label(self.add_panel, text="新增订阅:",
                         font=(None, 10)).pack(pady=20)
        self.new_author_entry = tk.Entry(self.add_panel)
        self.new_author_entry.pack()
        confirm_button = tk.Button(self.add_panel,
                                   text="确认",
                                   font=(None, 10),
                                   command=self.check_new_author).pack(pady=20)
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()

    def check_new_author(self):
        """
        用户点击"确认"按钮后，检查输入是否合法
        
        """
        new_author = self.new_author_entry.get()
        if new_author not in self.all_authors:
            self.hint_label.config(text=f"目前仅支持订阅:{self.all_authors}")
        elif new_author in self.reader.authors:
            self.hint_label.config(text="该作者已订阅，请重新输入")
        else:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk,
                      text="确认",
                      font=(None, 12),
                      command=tmp_tk.destroy).pack(pady=20)
            self.reader.subscribeToPublisher(new_author, self.publisher)
            self.add_panel.destroy()
            self.update_main_panel()

    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅",
                              font=(None, 12)).pack(pady=20)
        for author in self.reader.authors:
            button = tk.Button(self.root,
                               text=author,
                               width=13,
                               font=(None, 12),
                               command=lambda: self.show_titles(author))
            # button.bind("<Button-1>", self.show_titles)
            button.pack(pady=10)
        self.add_button = tk.Button(
            self.root,
            text="新增订阅",
            font=(None, 12),
            command=self.add_subscription).pack(pady=40)
        self.root.update()

    def show_titles(self, author):
        """
        用户点击主界面的“作者名”按钮：
        
        1. 显示该目前接收到的该作者的全部诗的标题，要求滚动条显示（tkinter的Scrollbar控件）
        
        2. 点击标题，显示诗的具体内容（提示：可以使用 Listbox 控件的bind方法，动作为 <ListboxSelect>)
        
        3. 选做：增加诗题检索功能，获取标题符合条件的诗
        
        
        Notice：
        
        1. Demo 仅为参照，布局可按自己的喜好设计
        
        2. 选做题的检索方式自定，全字匹配、正则检索等都可以
        
        3. 选做题不强制，占 1-2 分，不想做检索也可设计其他功能
        
        """
        # TODO
        # print(self.publisher)
        # print(author)
        self.show_titles_label = tk.Tk()
        self.show_titles_label.title(author)
        self.show_titles_label.geometry('400x300')
        # author_label = tk.Label(self.show_titles_label,
        #                         text=author,
        #                         font=(None, 10)).pack(pady=20)
        title_list_total = [i[0] for i in self.reader.poems[author]]
        # select_list=list(range(len(title_list_total)))
        search_entry = tk.Entry(self.show_titles_label)
        search_entry.pack()
        select_list = []

        def search(*args):
            title_listbox.delete(0, len(select_list))
            select_list.clear()
            print(search_entry.get())
            #TODO!!!
            pattern=search_entry.get()
            for i in range(len(title_list_total)):
                string=title_list_total[i]
                if re.search(pattern,string):
                    select_list.append(i)
            for i in range(len(select_list)):
                title_listbox.insert(i, title_list_total[select_list[i]])

        button_label = tk.Label(self.show_titles_label)

        search_button = tk.Button(button_label, text='搜索',
                                  command=search).pack(side='right')

        def back(*args):
            nonlocal select_list
            # print(select_list)
            title_listbox.delete(0, len(select_list))
            select_list.clear()
            select_list += list(range(len(title_list_total)))
            for i in range(len(select_list)):
                title_listbox.insert(i, title_list_total[select_list[i]])

        back_button = tk.Button(button_label, text='后退',
                                command=back).pack(side='left')
        button_label.pack()
        select_list = list(range(len(title_list_total)))

        scrollbar = tk.Scrollbar(self.show_titles_label)
        scrollbar.pack(side='right', fill='y')
        title_listbox = tk.Listbox(self.show_titles_label)
        title_listbox.config(yscrollcommand=scrollbar.set)

        def show_poem(*args):
            if not title_listbox.curselection():
                return
            id = select_list[title_listbox.curselection()[0]]
            poem_label = tk.Tk()
            poem_label.geometry('150x150')
            poem_label.title(title_list_total[id])
            poem_text = tk.Text(poem_label)

            print(self.reader.poems[author][id])
            for i in range(4):
                poem_text.insert(
                    str(i + 1) + '.0',
                    self.reader.poems[author][id][i + 2] + '\n')
            poem_text.pack()

        title_listbox.bind('<<ListboxSelect>>', show_poem)
        title_listbox.pack()
        for i in range(len(select_list)):
            title_listbox.insert(i, title_list_total[select_list[i]])
        scrollbar.config(command=title_listbox.yview)

        # print(title_list_total)

